#### Problem
Map-reduce operations are essential for efficient task decomposition and parallel processing.

It has two phases:

(1) Map - Break a task into smaller sub-tasks, processing each sub-task in parallel.

(2) Reduce - Aggregate the results across all of the completed, parallelized sub-tasks.

Let's design a system that will do two things:

(1) Map - Create a set of jokes about a topic.

(2) Reduce - Pick the best joke from the list.

We'll use an LLM to do the job generation and selection.

In [1]:
from langchain_openai import ChatOpenAI

# Prompts we will use
subjects_prompt = """Generate a list of 3 sub-topics that are all related to this overall topic: {topic}."""
joke_prompt = """Generate a joke about {subject}"""
best_joke_prompt = """Below are a bunch of jokes about {topic}. Select the best one! Return the ID of the best one, starting 0 as the ID for the first joke. Jokes: \n\n  {jokes}"""

# LLM
model = ChatOpenAI(model="gpt-4o-mini", temperature=0) 

#### State

In [2]:
import operator
from typing import Annotated
from typing_extensions import TypedDict
from pydantic import BaseModel

class Subjects(BaseModel):
    subjects: list[str]

class BestJoke(BaseModel):
    id: int
    
class OverallState(TypedDict):
    topic: str
    subjects: list
    jokes: Annotated[list, operator.add]
    best_selected_joke: str

In [3]:
def generate_topics(state: OverallState):
    prompt = subjects_prompt.format(topic=state["topic"])
    response = model.with_structured_output(Subjects).invoke(prompt)
    return {"subjects": response.subjects}

In [4]:
from langgraph.types import Send

def continue_to_jokes(state: OverallState):
    return [Send("generate_joke", {"subject": s}) for s in state["subjects"]]

In [ ]:
# Joke Generation

class JokeState(TypedDict):
    subject: str

class Joke(BaseModel):
    joke: str

def generate_joke(state: JokeState):
    prompt = joke_prompt.format(subject=state["subject"])
    response = model.with_structured_output(Joke).invoke(prompt)
    return {"jokes": [response.joke]}

In [7]:
def best_joke(state: OverallState):
    jokes = "\n\n".join(state["jokes"])
    prompt = best_joke_prompt.format(topic=state["topic"], jokes=jokes)
    response = model.with_structured_output(BestJoke).invoke(prompt)
    return {"best_selected_joke": state["jokes"][response.id]}

#### Compile

In [ ]:
from IPython.display import Image
from langgraph.graph import END, StateGraph, START

# Construct the graph: here we put everything together to construct our graph
graph = StateGraph(OverallState)

graph.add_node("generate_topics", generate_topics)
graph.add_node("generate_joke", generate_joke)
graph.add_node("best_joke", best_joke)

graph.add_edge(START, "generate_topics")
graph.add_conditional_edges("generate_topics", continue_to_jokes, ["generate_joke"])

graph.add_edge("generate_joke", "best_joke")
graph.add_edge("best_joke", END)

# Compile the graph
app = graph.compile()
# Image(app.get_graph().draw_mermaid_png())
app.get_graph().print_ascii()


   +-----------+     
   | __start__ |     
   +-----------+     
          *          
          *          
          *          
+-----------------+  
| generate_topics |  
+-----------------+  
          .          
          .          
          .          
 +---------------+   
 | generate_joke |   
 +---------------+   
          *          
          *          
          *          
   +-----------+     
   | best_joke |     
   +-----------+     
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+      


In [11]:
# Call the graph: here we call it to generate a list of jokes
for s in app.stream({"topic": "animals"}):
    print(s)

{'generate_topics': {'subjects': ['Animal Behavior', 'Endangered Species Conservation', 'Animal Habitats and Ecosystems']}}
{'generate_joke': {'jokes': ['Why did the endangered species start a band? \n\nBecause they wanted to raise awareness and hit all the right notes for conservation!']}}
{'generate_joke': {'jokes': ["Why did the fish blush? \n\nBecause it saw the ocean's bottom!"]}}
{'generate_joke': {'jokes': ['Why did the chicken join a band? \n\nBecause it had the drumsticks!']}}
{'best_joke': {'best_selected_joke': 'Why did the chicken join a band? \n\nBecause it had the drumsticks!'}}
